In [1]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Tracking", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)
            
            # Get the landmarks for the thumb
            thumb_tip = handLms.landmark[4]
            thumb_base = handLms.landmark[2]

            # Check if the thumb is pointing up (tip is above the base)
            if thumb_tip.y < thumb_base.y:
                cv2.putText(img, "Thumb Up Detected", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Hand Tracking", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import subprocess  # To open applications

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize mediapipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Path to your app (example: opening Chrome browser)
app_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)

            # Get the landmarks for the thumb
            thumb_tip = handLms.landmark[4]
            thumb_base = handLms.landmark[2]

            # Check if the thumb is pointing up (thumb tip is above the base)
            if thumb_tip.y < thumb_base.y:
                cv2.putText(img, "Thumb Up Detected", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                # Trigger app launch (if thumb-up is detected)
                subprocess.Popen([app_path])  # This will open the app

    # Display the result
    cv2.imshow("Hand Tracking", img)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
import subprocess
from tkinter import *
import threading

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Define gesture -> app mapping
GESTURE_ACTIONS = {
    "fist": "notepad",     # Fist will open Notepad
    "peace": "calc",       # Peace sign will open Calculator
}

def detect_gesture(landmarks):
    # Very simple gesture recognition using landmark distance
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]

    if abs(index_tip.y - middle_tip.y) < 0.02:
        return "peace"
    elif abs(index_tip.y - thumb_tip.y) < 0.02:
        return "fist"
    else:
        return None

def run_camera():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process hand landmarks
        result = hands.process(rgb)
        gesture = None

        if result.multi_hand_landmarks:
            
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks.landmark)

        # Display gesture and open app
        if gesture:
            cv2.putText(frame, f"Gesture: {gesture}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if gesture in GESTURE_ACTIONS:
                subprocess.Popen(GESTURE_ACTIONS[gesture], shell=True)

        cv2.imshow("Hand Sign Recognition", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
            break

    cap.release()
    cv2.destroyAllWindows()


In [2]:
def start_detection():
    threading.Thread(target=run_camera).start()

root = Tk()
root.title("Hand Sign App Launcher")

label = Label(root, text="Click to Start Gesture Detection", font=("Arial", 16))
label.pack(pady=20)

start_button = Button(root, text="Start", command=start_detection, font=("Arial", 14))
start_button.pack(pady=10)

root.mainloop()


In [ ]:
import cv2
import mediapipe as mp
import subprocess
from tkinter import *
import threading
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Gesture-to-app mapping
GESTURE_ACTIONS = {
    "fist": "notepad",
    "peace": "calc",
    "palm": "C:/Program Files/Google/Chrome/Application/chrome.exe",
    "thumbs_up": "mspaint",
    "ok_sign": "explorer"
}

# Cooldown tracking (seconds)
last_launched = {gesture: 0 for gesture in GESTURE_ACTIONS}
COOLDOWN_SECONDS = 10  # Prevent repeat within 10 seconds

# Detect gesture using hand landmarks
def detect_gesture(landmarks):
    tips = [landmarks[i] for i in [4, 8, 12, 16, 20]]  # Thumb to pinky tips
    bases = [landmarks[i] for i in [3, 6, 10, 14, 18]] # Finger bases

    def is_up(tip, base):
        return tip.y < base.y

    fingers = [is_up(tips[i], bases[i]) for i in range(5)]

    if fingers == [False, False, False, False, False]:
        return "fist"
    elif fingers == [False, True, True, False, False]:
        return "peace"
    elif fingers == [True, True, True, True, True]:
        return "palm"
    elif fingers == [True, False, False, False, False]:
        return "thumbs_up"
    elif fingers[0] and fingers[1] and abs(landmarks[4].x - landmarks[8].x) < 0.05:
        return "ok_sign"
    else:
        return None

# Camera + gesture detection
def run_camera():
    cap = cv2.VideoCapture(0)
    last_gesture = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        gesture = None

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks.landmark)

        current_time = time.time()

        if gesture and gesture != last_gesture:
            last_gesture = gesture
            if gesture in GESTURE_ACTIONS:
                if current_time - last_launched[gesture] > COOLDOWN_SECONDS:
                    print(f"Launching {gesture} → {GESTURE_ACTIONS[gesture]}")
                    subprocess.Popen(GESTURE_ACTIONS[gesture], shell=True)
                    last_launched[gesture] = current_time
                else:
                    print(f"Gesture '{gesture}' is on cooldown.")
        elif gesture is None:
            last_gesture = None

        cv2.putText(frame, f"Gesture: {gesture or 'None'}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Sign Recognition", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# GUI
def start_detection():
    threading.Thread(target=run_camera).start()

root = Tk()
root.title("Hand Sign App Launcher")
root.geometry("400x200")

label = Label(root, text="Click to Start Gesture Recognition", font=("Arial", 14))
label.pack(pady=20)

start_button = Button(root, text="Start", command=start_detection, font=("Arial", 14), bg="green", fg="white")
start_button.pack(pady=10)

root.mainloop()


Launching thumbs_up → mspaint
Gesture 'thumbs_up' is on cooldown.
Gesture 'thumbs_up' is on cooldown.
Gesture 'thumbs_up' is on cooldown.
Launching palm → C:/Program Files/Google/Chrome/Application/chrome.exe
Gesture 'thumbs_up' is on cooldown.


In [1]:
import cv2
import mediapipe as mp
import subprocess
from tkinter import *
import threading
import time
from PIL import Image, ImageTk

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Gesture-to-app mapping
GESTURE_ACTIONS = {
    "fist": "C:\Games\Adobe Premiere Pro 2025",
    "peace": "calc",
    "palm": "C:\Program Files\Google\Chrome\Application\chrome.exe",
    "thumbs_up": "C:\Games\Forza Horizon 5\Forza Horizon5.exe",
    "ok_sign": "explorer"
}

# Cooldown tracking (seconds)
last_launched = {gesture: 0 for gesture in GESTURE_ACTIONS}
COOLDOWN_SECONDS = 10  # Prevent repeat within 10 seconds

# Detect gesture using hand landmarks
def detect_gesture(landmarks):
    tips = [landmarks[i] for i in [4, 8, 12, 16, 20]]  # Thumb to pinky tips
    bases = [landmarks[i] for i in [3, 6, 10, 14, 18]] # Finger bases

    def is_up(tip, base):
        return tip.y < base.y

    fingers = [is_up(tips[i], bases[i]) for i in range(5)]

    if fingers == [False, False, False, False, False]:
        return "fist"
    elif fingers == [False, True, True, False, False]:
        return "peace"
    elif fingers == [True, True, True, True, True]:
        return "palm"
    elif fingers == [True, False, False, False, False]:
        return "thumbs_up"
    elif fingers[0] and fingers[1] and abs(landmarks[4].x - landmarks[8].x) < 0.05:
        return "ok_sign"
    else:
        return None

# Camera + gesture detection
def run_camera(canvas, window):
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    last_gesture = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        gesture = None

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks.landmark)

        current_time = time.time()

        if gesture and gesture != last_gesture:
            last_gesture = gesture
            if gesture in GESTURE_ACTIONS:
                if current_time - last_launched[gesture] > COOLDOWN_SECONDS:
                    print(f"Launching {gesture} → {GESTURE_ACTIONS[gesture]}")
                    subprocess.Popen(GESTURE_ACTIONS[gesture], shell=True)
                    last_launched[gesture] = current_time
                else:
                    print(f"Gesture '{gesture}' is on cooldown.")
        elif gesture is None:
            last_gesture = None

        # Convert the frame to Tkinter format and display it on the canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert color for display
        img = Image.fromarray(frame)  # Convert to PIL Image
        img_tk = ImageTk.PhotoImage(image=img)  # Convert to ImageTk format

        # Update the canvas with the new image
        canvas.create_image(0, 0, anchor=NW, image=img_tk)
        window.update_idletasks()

    cap.release()

# GUI Setup
def start_detection():
    threading.Thread(target=run_camera, args=(canvas, window), daemon=True).start()

# Tkinter GUI
window = Tk()
window.title("Hand Sign App Launcher")
window.geometry("1280x720")  # Make window larger

label = Label(window, text="Click to Start Gesture Recognition", font=("Arial", 20))
label.pack(pady=30)

start_button = Button(window, text="Start", command=start_detection, font=("Arial", 18), bg="green", fg="white", width=15, height=2)
start_button.pack(pady=20)

# Create a canvas to display the camera feed
canvas = Canvas(window, width=1280, height=720)
canvas.pack()

window.mainloop()


Launching thumbs_up → C:\Games\Forza Horizon 5\Forza Horizon5.exe
Gesture 'thumbs_up' is on cooldown.
Launching fist → C:\Games\Adobe Premiere Pro 2025


In [8]:
import cv2
import mediapipe as mp
import subprocess
from tkinter import *
import threading
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Gesture-to-app mapping with corrected paths
GESTURE_ACTIONS = {
    "fist": r"C:/Games/Adobe Premiere Pro 2025/Adobe Premiere Pro.exe",
    "victory": "calc",  # Using simple app for testing
    "palm": r"C:/Program Files/Google/Chrome/Application/chrome.exe",
    "thumbs_up": r"C:/Games/Forza Horizon 5/Forza Horizon5.exe",
    "rock_on": "explorer"
}

# Cooldown tracking (seconds)
last_launched = {gesture: 0 for gesture in GESTURE_ACTIONS}
COOLDOWN_SECONDS = 10  # Prevent repeat within 10 seconds

# Variable to control the camera loop
camera_running = False

# Detect gesture using hand landmarks
def detect_gesture(landmarks):
    tips = [landmarks[i] for i in [4, 8, 12, 16, 20]]  # Thumb to pinky tips
    bases = [landmarks[i] for i in [3, 6, 10, 14, 18]] # Finger bases

    def is_up(tip, base):
        return tip.y < base.y

    fingers = [is_up(tips[i], bases[i]) for i in range(5)]

    if fingers == [False, False, False, False, False]:
        return "fist"
    elif fingers == [False, True, True, False, False]:
        return "victory"
    elif fingers == [True, True, True, True, True]:
        return "palm"
    elif fingers == [True, False, False, False, False]:
        return "thumbs_up"
    elif fingers[0] and fingers[1] and abs(landmarks[4].x - landmarks[8].x) < 0.05:
        return "rock_on"
    else:
        return None

# Camera + gesture detection
def run_camera():
    global camera_running
    cap = cv2.VideoCapture(0)
    last_gesture = None

    while camera_running:  # Loop will run while camera_running is True
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        gesture = None

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks.landmark)

        current_time = time.time()

        if gesture and gesture != last_gesture:
            last_gesture = gesture
            if gesture in GESTURE_ACTIONS:
                if current_time - last_launched[gesture] > COOLDOWN_SECONDS:
                    print(f"Launching {gesture} → {GESTURE_ACTIONS[gesture]}")
                    launch_app(GESTURE_ACTIONS[gesture])
                    last_launched[gesture] = current_time
                else:
                    print(f"Gesture '{gesture}' is on cooldown.")
        elif gesture is None:
            last_gesture = None

        cv2.putText(frame, f"Gesture: {gesture or 'None'}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Sign Recognition", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Break if 'Esc' key is pressed
            break

    cap.release()
    cv2.destroyAllWindows()

# Launch application
def launch_app(app_path):
    try:
        subprocess.Popen("C:/Games/Adobe Premiere Pro 2025/Adobe Premiere Pro.exe", shell=True)
        print(f"Launching {"C:/Games/Adobe Premiere Pro 2025/Adobe Premiere Pro.exe"}")
    except Exception as e:
        print(f"Error launching {"C:/Games/Forza Horizon 5/Forza Horizon5.exe"}: {e}")

# Start camera detection
def start_detection():
    global camera_running
    camera_running = True  # Set flag to True to start camera
    threading.Thread(target=run_camera, daemon=True).start()

# Stop camera detection
def stop_detection():
    global camera_running
    camera_running = False  # Set flag to False to stop camera
    print("Camera stopped.")

# GUI
root = Tk()
root.title("Hand Sign App Launcher")
root.geometry("400x200")

label = Label(root, text="Click to Start Gesture Recognition", font=("Arial", 14))
label.pack(pady=20)

# Start Button
start_button = Button(root, text="Start", command=start_detection, font=("Arial", 14), bg="green", fg="white")
start_button.pack(pady=10)

# Stop Button
stop_button = Button(root, text="Stop", command=stop_detection, font=("Arial", 14), bg="red", fg="white")
stop_button.pack(pady=10)

root.mainloop()


SyntaxError: f-string: expecting '}' (4164053277.py, line 101)

In [11]:
import cv2
import mediapipe as mp
import subprocess
from tkinter import *
import threading
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Gesture-to-app mapping with corrected paths
GESTURE_ACTIONS = {
    "fist": r"C:/Games/Adobe Premiere Pro 2025/Adobe Premiere Pro.exe",
    "victory": "calc",  # Calculator for testing
    "palm": r"C:/Program Files/Google/Chrome/Application/chrome.exe",
    "thumbs_up": r"C:\Games\Clair Obscur - Expedition 33\Expedition33_Steam.exe",
    "rock_on": "explorer"
}

# Cooldown tracking (seconds)
last_launched = {gesture: 0 for gesture in GESTURE_ACTIONS}
COOLDOWN_SECONDS = 10  # Prevent repeat within 10 seconds

# Variable to control the camera loop
camera_running = False

# Detect gesture using hand landmarks
def detect_gesture(landmarks):
    tips = [landmarks[i] for i in [4, 8, 12, 16, 20]]  # Thumb to pinky tips
    bases = [landmarks[i] for i in [3, 6, 10, 14, 18]]  # Finger bases

    def is_up(tip, base):
        return tip.y < base.y

    fingers = [is_up(tips[i], bases[i]) for i in range(5)]

    if fingers == [False, False, False, False, False]:
        return "fist"
    elif fingers == [False, True, True, False, False]:
        return "victory"
    elif fingers == [True, True, True, True, True]:
        return "palm"
    elif fingers == [True, False, False, False, False]:
        return "thumbs_up"
    elif fingers[0] and fingers[1] and abs(landmarks[4].x - landmarks[8].x) < 0.05:
        return "rock_on"
    else:
        return None

# Launch application safely
def launch_app(app_path):
    try:
        subprocess.Popen(app_path, shell=True)
        print(f"Launching {app_path}")
    except Exception as e:
        print(f"Error launching {app_path}: {e}")

# Camera + gesture detection
def run_camera():
    global camera_running
    cap = cv2.VideoCapture(0)
    last_gesture = None

    while camera_running:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        gesture = None

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks.landmark)

        current_time = time.time()

        if gesture and gesture != last_gesture:
            last_gesture = gesture
            if gesture in GESTURE_ACTIONS:
                if current_time - last_launched[gesture] > COOLDOWN_SECONDS:
                    print(f"Detected gesture: {gesture}")
                    launch_app(GESTURE_ACTIONS[gesture])
                    last_launched[gesture] = current_time
                else:
                    print(f"Gesture '{gesture}' is on cooldown.")
        elif gesture is None:
            last_gesture = None

        cv2.putText(frame, f"Gesture: {gesture or 'None'}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Sign Recognition", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Esc key to stop
            break

    cap.release()
    cv2.destroyAllWindows()

# Start camera detection
def start_detection():
    global camera_running
    camera_running = True
    threading.Thread(target=run_camera, daemon=True).start()

# Stop camera detection
def stop_detection():
    global camera_running
    camera_running = False
    print("Camera stopped.")

# GUI
root = Tk()
root.title("Hand Sign App Launcher")
root.geometry("400x200")

label = Label(root, text="Click to Start Gesture Recognition", font=("Arial", 14))
label.pack(pady=20)

start_button = Button(root, text="Start", command=start_detection, font=("Arial", 14), bg="green", fg="white")
start_button.pack(pady=10)

stop_button = Button(root, text="Stop", command=stop_detection, font=("Arial", 14), bg="red", fg="white")
stop_button.pack(pady=10)

root.mainloop()


Detected gesture: palm
Launching C:/Program Files/Google/Chrome/Application/chrome.exe
Detected gesture: rock_on
Launching explorer
Gesture 'rock_on' is on cooldown.
Gesture 'rock_on' is on cooldown.
Gesture 'rock_on' is on cooldown.
Gesture 'rock_on' is on cooldown.
Detected gesture: rock_on
Launching explorer
Gesture 'rock_on' is on cooldown.
Detected gesture: thumbs_up
Launching C:\Games\Clair Obscur - Expedition 33\Expedition33_Steam.exe
Camera stopped.
